PyTorch Hubを利用します
* https://pytorch.org/docs/stable/hub.html
* https://pytorch.org/hub/

今回使うのはPytorch Teamが公開するMobileNet v2のモデルです
* https://pytorch.org/hub/pytorch_vision_mobilenet_v2/

サンプルコードはBSD-3-Clauseライセンスで公開されている以下のサンプルを参照しています。  
https://github.com/pytorch/pytorch.github.io/blob/site/assets/hub/pytorch_vision_mobilenet_v2.ipynb
`Copyright (c) 2018, Facebook Inc`  
https://github.com/pytorch/pytorch.github.io/blob/site/LICENSE

In [1]:
import torchvision
print(torchvision.__version__)

/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


0.13.0


In [2]:
import torch
print(torch.__version__)

1.12.0


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")  
else:
    device = torch.device("cpu")
print(device)

cpu


In [4]:
# モデルをダウンロードする
model = torch.hub.load('pytorch/vision', 'mobilenet_v2', pretrained=True)

Downloading: "https://github.com/pytorch/vision/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [5]:
# モデルを読み込む
model.to(device)
model.eval()

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [8]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://raw.githubusercontent.com/pytorch/hub/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [9]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([ 2.0977e+00, -1.7348e+00, -2.2355e+00, -2.9669e+00, -2.3805e+00,
         9.7398e-01, -1.6049e+00,  3.6914e+00,  6.3812e+00, -1.2929e+00,
        -6.7555e+00, -3.3525e+00, -7.9619e+00, -4.4554e+00, -5.6423e+00,
        -4.6624e+00, -1.9577e+00, -3.5811e-01, -1.2812e+00, -4.6707e+00,
        -3.2935e+00, -2.5674e+00, -2.4351e+00, -1.3017e+00, -3.2453e+00,
        -1.4237e+00, -1.2001e+00,  4.1275e-01, -1.6093e+00,  1.5871e+00,
         2.7725e-01, -6.2651e-01, -2.9734e-01, -3.8219e+00, -1.5450e+00,
        -2.8976e+00, -5.6528e-01, -2.3938e+00, -3.3704e-01,  1.2809e+00,
        -1.2516e+00, -2.6469e+00, -3.1011e+00, -2.2447e+00, -4.4385e-01,
        -1.2620e+00,  8.2895e-01, -2.0436e+00, -6.6037e-01, -8.6518e-02,
         4.8967e-01, -1.7190e+00, -7.7943e-01, -1.1046e+00, -5.3857e-01,
        -2.9254e+00, -1.9327e+00, -2.7273e+00, -6.0903e-01, -1.6802e+00,
         1.3443e+00, -4.2062e+00, -1.4768e+00, -4.5581e+00, -3.2726e+00,
        -4.0086e+00,  1.5702e-01, -1.9921e+00, -7.4

In [11]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2022-08-01 02:30:23--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.001s  

2022-08-01 02:30:23 (10.8 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [12]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Samoyed 0.8303049206733704
Pomeranian 0.06988745182752609
keeshond 0.012964102439582348
collie 0.010797764174640179
Great Pyrenees 0.009886696003377438


In [15]:
%matplotlib inline

In [14]:
# 結果可視化用にCOCOのアノテーションラベルリストを取得
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')
classes_to_labels = utils.get_coco_object_dictionary()

# 結果可視化用関数定義
from matplotlib import pyplot as plt
import matplotlib.patches as patches

# The utility plots the images and predicted bounding boxes (with confidence scores).
def plot_results(best_results):
    for image_idx in range(len(best_results)):
        fig, ax = plt.subplots(1)
        # Show original, denormalized image...
        image = inputs[image_idx] / 2 + 0.5
        ax.imshow(image)
        # ...with detections
        bboxes, classes, confidences = best_results[image_idx]
        for idx in range(len(bboxes)):
            left, bot, right, top = bboxes[idx]
            x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
            rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
    plt.show()

/usr/local/lib/python3.8/dist-packages/torch/hub.py:266: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /root/.cache/torch/hub/torchhub.zip
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(

In [16]:
# 使用する画像リストを作成（ローカルにある画像へのパスを指定しても可）
uris = [
    'http://images.cocodataset.org/val2017/000000397133.jpg',
    'http://images.cocodataset.org/val2017/000000037777.jpg',
    'http://images.cocodataset.org/val2017/000000252219.jpg',
    'https://rt-net.jp/wp-content/uploads/2020/10/edu_seminar_image_raspimouse.png-600x400.jpg',
    'https://rt-net.jp/mobility/wp-content/uploads/2022/03/s-IMG_2553_b.jpg',
    'https://ultralytics.com/images/zidane.jpg'
]

In [17]:
# 画像取得しネットワークへ入力できるように変換
inputs = [utils.prepare_input(uri) for uri in uris]
tensor = utils.prepare_tensor(inputs)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# 物体検出
with torch.no_grad():
    detections_batch = ssd_model(tensor)

results_per_input = utils.decode_results(detections_batch)
best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]

In [ ]:
# 結果可視化
plot_results(best_results_per_input)